In [ ]:

def get_derangements(G):
    Drep = []
    D = []
    CC = G.conjugacy_classes_representatives()
    for x in CC:
        if is_derangement(G,x) == True:
            Drep.append(x)
        else:
            pass
    for x in Drep:
          D += list(G.conjugacy_class(x))
    return D
    #return filter(is_derangement,G)
def is_derangement(G,x):  # needs G to be defined beforehand
    for i in G.domain():
        if x(i) == i:
            return False
        else:
            pass
    return True

def eigenvalues_group(G):
    IRR = G.irreducible_characters()
    derangement_conjugacy_classes_representatives = []
    eigenvalues = []
    CC = G.conjugacy_classes_representatives()
    for x in CC:
        if is_derangement(G,x) == True:
            derangement_conjugacy_classes_representatives.append(x)
    for phi in IRR:
        s = 0
        for x in derangement_conjugacy_classes_representatives:
            s += (1/phi.degree()) * ( G.conjugacy_class(x).cardinality()*phi(x) )
        eigenvalues.append(s)
    return list(eigenvalues),gap.Minimum(eigenvalues),gap.Maximum(eigenvalues)

def eigenvalues_group_and_multi(G):
    IRR = G.irreducible_characters()
    derangement_conjugacy_classes_representatives = []
    eigenvalues, multiplicity = [], []
    CC = G.conjugacy_classes_representatives()
    for x in CC:
        if is_derangement(G,x) == True:
            derangement_conjugacy_classes_representatives.append(x)
    for phi in IRR:
        s = 0
        for x in derangement_conjugacy_classes_representatives:
            s += (1/phi.degree()) * ( G.conjugacy_class(x).cardinality()*phi(x) )
        eigenvalues.append(s)
        multiplicity.append(phi(G.identity())^2)
    return eigenvalues,multiplicity

def eigenvalues_group_and_characters(G):
    IRR = G.irreducible_characters()
    derangement_conjugacy_classes_representatives = []
    eigenvalues_characters = []
    CC = G.conjugacy_classes_representatives()
    for x in CC:
        if is_derangement(G,x) == True:
            derangement_conjugacy_classes_representatives.append(x)
    for phi in IRR:
        s = 0
        for x in derangement_conjugacy_classes_representatives:
            s += (1/phi.degree()) * ( G.conjugacy_class(x).cardinality()*phi(x) )
        eigenvalues_characters.append([s,phi.values()])
    return list(eigenvalues_characters)

def derangement_graph(G):
    D = get_derangements(G)
    X = G.cayley_graph(generators = D)
    X = Graph(X)
    return X

#def test_graph(L):


def test_graphs6():
    L = graph6_to_list()
    LL = [Graph(L[i]) for i in range(len(L))]
    print [LL[i].automorphism_group().order() for i in range(len(LL))]
    #LLL = divide_lists(LL,100)
    f = open("output4_graph6-jup.txt","w+")
    g = open("output4_graph6_3000-jup.txt","w+")
    h = open("output4_graph6_not_EKR-jup.txt","w+")
    for i in range(len(LL)):
        X = LL[i]
        X.relabel()
        G = X.automorphism_group()
        if G.order() < 3500:
            Y = derangement_graph(G)
            independent_number = Y.independent_set(value_only = True)
            #clique_number = Y.clique_number()
            evalue_multiplicity = eigenvalues_group_and_multi(G)
            eigenvalues = list(evalue_multiplicity[0])
            multiplicity = list(evalue_multiplicity[1])
            evalue_min_max = eigenvalues_group(G)
            print "----------------------------------------------------------"
            print "\t No",int(i+1)
            print "\n group =",gap.StructureDescription(G)
            print " Size = ",G.order()
            print " Domain = ",G.domain()
            print " Independence number = ",independent_number, "\n stabilizer = ",G.stabilizer(1).order()
            print " EKR",G.stabilizer(1).order() == independent_number
            print " eigenvalues",eigenvalues
            print " multiplicity", multiplicity
            print " Blocks",G.blocks_all()
            print " Join", evalue_min_max[2]-evalue_min_max[1] == G.order()
            print " ratio-bound", (G.order())/(1-(evalue_min_max[2]/evalue_min_max[1]))
            f.write("----------------------------------------------------------------------------- \n \t No {0}\n graph = {12} \n group = {1}\n Size = {2}\n Degree = {6} \n EKR {3}\n eigenvalues = {4}\n multiplicity = {7} \n Blocks = {5}\n Join {8}\n ratio-bound = {9}\n independent_number = {10}\n stabilizer = {11}            \n----------------------------------------------------------------------------- \n".format(int(i),gap.StructureDescription(G),G.order(),G.stabilizer(1).order() == independent_number,eigenvalues,G.blocks_all(),G.degree(),multiplicity,evalue_min_max[2]-evalue_min_max[1] ==G.order(),(G.order())/(1-(evalue_min_max[2]/evalue_min_max[1])),independent_number,G.stabilizer(1).order(),L[i]))
            if G.stabilizer(1).order() != independent_number:
                print "no-ekr"
                h.write("\n ------------------------- \n ")
                h.write("Index = {1}\n graph-group{0} \n \n".format([L[i],G.gens()],i))
                h.write("\n ------------------------- \n ")
        else:
            print "aaa"
            g.write("Index = {0} \n Group = {1} \n Generators = {2} \n Graph = {3}\n ---------------------------------- \n ".format(i,gap.StructureDescription(G),G.gens(),L[i]))
    print "----------------------------------------------------------"
    f.close()
    h.close()
    g.close


        #print "-----------------------------+"
        #print "\n Independence number = ",independent_number, "\n stabilizer = ",G.stabilizer(1).order(),"\n transitive", X.is_vertex_transitive()#,"\n clique-coclique =", float(G.order()/clique_number)
        #print "ratio bound = ",ratio_bound(G)
        #return eigenvalues

def divide_lists(L,k):
    S = []
    s = len(L)%k
    q = (len(L)-s)/k
    for i in range(q):
        S.append([L[j] for j in range(i*k,i*k+k)])
    if s != 0 :
        S.append([L[i] for i in range(q*k,q*k+s)])
    return S

def graph6_to_list():
    L = [10,15,16,18,20,24,26,28,30]
    print L
    LL = []
    for i in range(len(L)):
        f = open("noncay{0}.g6".format(L[i]),"r")
        LL += f.readlines()
        f.close()
    LLL = []
    for i in range(len(LL)):
        LLL.append(LL[i].replace("\n",""))
    return LLL

test_graphs6()
P = graph6_to_list()
#print


 [10, 15, 16, 18, 20, 24, 26, 28, 30]
[120, 120, 120, 720, 720, 120, 32, 32, 32, 32, 32, 32, 32, 32, 144, 36, 36, 144, 240, 120, 28800, 320, 240, 120, 240, 320, 640, 160, 40, 240, 160, 320, 40, 60, 28800, 640, 240, 40, 40, 60, 240, 320, 40, 40, 40, 40, 400, 40, 40, 40, 40, 40, 40, 40, 40, 120, 40, 1440, 120, 40, 40, 40, 40, 120, 1440, 120, 40, 40, 40, 40, 400, 40, 40, 40, 40, 40, 240, 40, 320, 40, 60, 240, 640, 28800, 160, 40, 320, 60, 240, 640, 160, 40, 320, 120, 240, 320, 240, 28800, 120, 240, 72, 72, 72, 72, 240, 192, 72, 72, 240, 72, 72, 72, 192, 48, 192, 72, 72, 72, 72, 72, 72, 72, 192, 48, 48, 72, 72, 72, 72, 72, 72, 72, 192, 192, 48, 48, 48, 72, 72, 384, 192, 72, 72, 72, 48, 72, 48, 48, 48, 72, 48, 192, 48, 72, 72, 384, 72, 72, 72, 48, 48, 48, 384, 48, 48, 192, 72, 48, 72, 192, 48, 72, 48, 72, 72, 48, 192, 384, 72, 72, 72, 48, 192, 192, 72, 72, 72, 72, 72, 48, 72, 72, 72, 48, 192, 72, 72, 72, 72, 72, 240, 192, 72, 72, 72, 72, 240, 192, 72, 72, 72, 72, 52, 52, 52, 52, 52, 52, 52,

----------------------------------------------------------
	 No 5

 group = S6
 Size =  720
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
 Independence number =  48 
 stabilizer =  48
 EKR True
 eigenvalues [304, 64, 16, -32, 16, 16, -16, -16, -8, 16, 4]
 multiplicity [1, 1, 25, 25, 25, 25, 81, 81, 100, 100, 256]
 Blocks []
 Join False
 ratio-bound 480/7
----------------------------------------------------------
	 No 6

 group = S5
 Size =  120
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
 Independence number =  8 
 stabilizer =  8
 EKR True
 eigenvalues [64, 24, 4, -6, 0, -8, 4]
 multiplicity [1, 1, 16, 16, 25, 25, 36]
 Blocks [[0, 9, 10]]
 Join False
 ratio-bound 40/3
----------------------------------------------------------
	 No 7

 group = (C2 x C2 x C2) : C4
 Size =  32
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
 Independence number =  2 
 stabilizer =  2
 EKR True
 eigenvalues [29, -3, -3, -3, 1, 1, 1, 1, -3, 1, -1]
 multip

----------------------------------------------------------
	 No 23

 group = C2 x S5
 Size =  240
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
 Independence number =  12 
 stabilizer =  12
 EKR True
 eigenvalues [174, -6, -6, -66, -6, -6, -6, -6, -6, 6, -6, 6, 4, 4]
 multiplicity [1, 1, 1, 1, 16, 16, 16, 16, 25, 25, 25, 25, 36, 36]
 Blocks [[0, 1], [0, 5, 6, 7, 8, 9, 10, 11, 12, 13]]
 Join True
 ratio-bound 66
----------------------------------------------------------
	 No 24

 group = C2 x A5
 Size =  120
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
 Independence number =  6 
 stabilizer =  6
 EKR True
 eigenvalues [84, -6, 4, 4, -6, -6, -6, -6, 0, 6]
 multiplicity [1, 1, 9, 9, 9, 9, 16, 16, 25, 25]
 Blocks [[0, 1]]
 Join False
 ratio-bound 8
----------------------------------------------------------
	 No 25

 group = C2 x S5
 Size =  240
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 

----------------------------------------------------------
	 No 41

 group = C2 x S5
 Size =  240
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
 Independence number =  12 
 stabilizer =  12
 EKR True
 eigenvalues [174, -6, -6, -66, -6, -6, -6, -6, -6, 6, -6, 6, 4, 4]
 multiplicity [1, 1, 1, 1, 16, 16, 16, 16, 25, 25, 25, 25, 36, 36]
 Blocks [[0, 1, 2, 3, 4, 5, 6, 8, 9, 10], [0, 7]]
 Join True
 ratio-bound 66
----------------------------------------------------------
	 No 42

 group = C2 x ((C2 x C2 x C2 x C2) : D10)
 Size =  320
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
 Independence number =  16 
 stabilizer =  16
 EKR True
 eigenvalues [219, -1, -1, 59, -1, -1, -21, -21, -1, -1, -5, -5, -1, -1, 3, 3, 3, 3, 15, -17]
 multiplicity [1, 1, 1, 1, 4, 4, 4, 4, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]
 Blocks [[0, 1, 2, 3], [0, 3]]
 Join False
 ratio-bound 28
---------------------------------------------------

----------------------------------------------------------
	 No 57

 group = C2 x (C5 : C4)
 Size =  40
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
 Independence number =  2 
 stabilizer =  2
 EKR True
 eigenvalues [34, -6, -6, -6, 4, 4, 4, 4, -1, -1]
 multiplicity [1, 1, 1, 1, 1, 1, 1, 1, 16, 16]
 Blocks [[0, 1], [0, 1, 2, 3], [0, 1, 10, 11, 12, 13, 14, 15, 16, 17], [0, 2], [0, 2, 4, 5, 10, 11, 16, 17, 18, 19], [0, 3], [0, 3, 6, 7, 8, 9, 10, 11, 16, 17], [0, 10, 11, 16, 17]]
 Join True
 ratio-bound 6
----------------------------------------------------------
	 No 58

 group = C2 x S6
 Size =  1440
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
 Independence number =  72 
 stabilizer =  72
 EKR True
 eigenvalues [954, -36, -36, 54, -18, 18, 0, -72, 0, -72, 18, -18, -6, -26, 4, 4, -18, 18, 0, 0, 9, 9]
 multiplicity [1, 1, 1, 1, 25, 25, 25, 25, 25, 25, 25, 25, 81, 81, 81, 81, 100, 100, 100, 100, 256, 256]
 Blocks [[

...

----------------------------------------------------------
	 No 305

 group = C13 : C4
 Size =  52
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}
 Independence number =  2 
 stabilizer =  2
 EKR True
 eigenvalues [38, -14, 12, 12, -1, -1, -1]
 multiplicity [1, 1, 1, 1, 16, 16, 16]
 Blocks [[0, 1], [0, 6, 7, 8, 9, 14, 15, 16, 17, 18, 19, 20, 21]]
 Join True
 ratio-bound 14
----------------------------------------------------------
	 No 306

 group = C13 : C4
 Size =  52
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}
 Independence number =  2 
 stabilizer =  2
 EKR True
 eigenvalues [38, -14, 12, 12, -1, -1, -1]
 multiplicity [1, 1, 1, 1, 16, 16, 16]
 Blocks [[0, 1, 2, 9, 10, 14, 15, 16, 17, 22, 23, 24, 25], [0, 13]]
 Join True
 ratio-bound 14
----------------------------------------------------------
	 No 307

 group = C13 : C4
 Size =  52
 Domain =  {0, 1, 2, 3, 4, 5, 6

----------------------------------------------------------
	 No 324

 group = C13 : C4
 Size =  52
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}
 Independence number =  2 
 stabilizer =  2
 EKR True
 eigenvalues [38, -14, 12, 12, -1, -1, -1]
 multiplicity [1, 1, 1, 1, 16, 16, 16]
 Blocks [[0, 1, 2, 5, 6, 11, 12, 15, 16, 21, 22, 23, 24], [0, 25]]
 Join True
 ratio-bound 14
----------------------------------------------------------
	 No 325

 group = C13 : C4
 Size =  52
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}
 Independence number =  2 
 stabilizer =  2
 EKR True
 eigenvalues [38, -14, 12, 12, -1, -1, -1]
 multiplicity [1, 1, 1, 1, 16, 16, 16]
 Blocks [[0, 1, 2, 3, 4, 13, 14, 15, 16, 20, 21, 24, 25], [0, 17]]
 Join True
 ratio-bound 14
----------------------------------------------------------
	 No 326

 group = C13 : C4
 Size =  52
 Domain =  {0, 1, 2, 3, 4, 5, 6

----------------------------------------------------------
	 No 343

 group = C13 : C4
 Size =  52
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}
 Independence number =  2 
 stabilizer =  2
 EKR True
 eigenvalues [38, -14, 12, 12, -1, -1, -1]
 multiplicity [1, 1, 1, 1, 16, 16, 16]
 Blocks [[0, 1, 2, 3, 4, 5, 6, 15, 16, 21, 22, 23, 24], [0, 25]]
 Join True
 ratio-bound 14
----------------------------------------------------------
	 No 344

 group = C13 : C4
 Size =  52
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}
 Independence number =  2 
 stabilizer =  2
 EKR True
 eigenvalues [38, -14, 12, 12, -1, -1, -1]
 multiplicity [1, 1, 1, 1, 16, 16, 16]
 Blocks [[0, 3, 4, 7, 8, 9, 10, 17, 18, 21, 22, 23, 24], [0, 25]]
 Join True
 ratio-bound 14
----------------------------------------------------------
	 No 345

 group = PSL(3,2) : C2
 Size =  336
 Domain =  {0, 1, 2, 3, 4, 5

----------------------------------------------------------
	 No 359

 group = PSL(3,2) : C2
 Size =  336
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27}
 Independence number =  12 
 stabilizer =  12
 EKR True
 eigenvalues [174, 6, 6, -8, -8, 6, -18, 6, 6]
 multiplicity [1, 1, 36, 36, 36, 49, 49, 64, 64]
 Blocks []
 Join False
 ratio-bound 63/2
----------------------------------------------------------
	 No 360

 group = PSL(3,2) : C2
 Size =  336
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27}
 Independence number =  12 
 stabilizer =  12
 EKR True
 eigenvalues [174, 6, 6, -8, -8, 6, -18, 6, 6]
 multiplicity [1, 1, 36, 36, 36, 49, 49, 64, 64]
 Blocks []
 Join False
 ratio-bound 63/2
----------------------------------------------------------
	 No 361

 group = PSL(3,2)
 Size =  168
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18

----------------------------------------------------------
	 No 376

 group = C2 x PSL(2,13)
 Size =  2184
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27}
 Independence number =  78 
 stabilizer =  78
 EKR True
 eigenvalues [1560, -78, 0, 0, -78, -78, 13, 13, 13, 13, 13, 13, -36, 6, 0, 0, 0, 0]
 multiplicity [1, 1, 49, 49, 49, 49, 144, 144, 144, 144, 144, 144, 169, 169, 196, 196, 196, 196]
 Blocks [[0, 27]]
 Join False
 ratio-bound 104
----------------------------------------------------------
	 No 377

 group = C2 x ((C2 x C2 x C2 x C2 x C2 x C2) : D14)
 Size =  1792
 Domain =  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27}
 Independence number =  64 
 stabilizer =  64
 EKR True
 eigenvalues [1245, -1, -1, 349, -1, -1, -1, -99, -99, -99, -1, -1, 61, -67, 1, 1, 3, 3, -1, -1, 9, 9, 5, 5, -13, -13, -3, -3, 1, 1, 1, 1, 7, 7, -5, -5, 3, 3, -1, -3]
 multiplicity [1, 1, 1